In [ ]:
import flopy
import os
import geopandas as gpd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import geopandas as gpd
import basic
import contextily as ctx
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import os
import matplotlib.pyplot as plt

import warnings

In [ ]:
import os
os.getcwd()

In [ ]:
m = basic.load_model()

In [ ]:
swr = gpd.read_file('GIS/SWR_Reaches.shp')

In [ ]:
sfr = gpd.read_file('GIS/nhd_hr_demo_sfr_cells.shp')

In [ ]:
# [lay, row, col, stage, cond]

In [ ]:
# get stream thalwegs to use as drain reach
swr__ = gpd.read_file("GIS/SWR_Reaches.shp")
min_elev = pd.read_csv(os.path.join('RR_2022/inputs/start_stage.tab'),sep = '\t', header = None)
min_elev.columns = ['reach', 'start_stage']
min_elev = pd.merge(min_elev, swr__.loc[:,['k','i','j','rno']], left_index = True, right_on = 'rno').drop(columns = 'rno')
min_elev = min_elev.rename(columns = {'start_stage':'modeltop'})
min_elev

In [ ]:
swr_active = False # set to true if SWR is active. this way no drain/swr cells will overlap

In [ ]:
stress_period = sfr.copy()

if swr_active:
    stress_period = stress_period.loc[~stress_period.node.isin(swr.node)]
    stress_period = stress_period.drop_duplicates(['i','j'])
else:
    stress_period = stress_period.loc[~stress_period.node.isin(swr.node)]
    stress_period = stress_period.drop_duplicates(['i','j'])

    stress_period = stress_period.append(min_elev)
    

stress_period.loc[:,'modeltop'] = m.dis.top[stress_period.loc[:,'i'], stress_period.loc[:,'j']]
stress_period.loc[:,'cond']  = 10000000
stress_period = stress_period.astype({'k':int, 'i':int, 'j':int})
stress_period = stress_period.loc[:,['k', 'i' , 'j', 'modeltop', 'cond']]
stress_period


In [ ]:
if stress_period.shape[0] == stress_period.drop_duplicates(['i','j']).shape[0]:
    print('there are no overlapping drain and SWR cells')
else:
    s = stress_period.shape[0] - stress_period.drop_duplicates(['i','j']).shape[0]
    warnings.warn(f'there ARE {s} overlapping drain and SWR cells')
    

In [ ]:
allsp = {x:(stress_period.values if x==0 else -1)  for x  in range(m.dis.nper-1)  }
allsp

In [ ]:
drn = flopy.modflow.mfdrn.ModflowDrn(m, stress_period_data= allsp)

In [ ]:
drn.write_file()
drn.plot(kper = 0, mflay  = 0)